In [170]:
import pandas as pd, numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
import keras
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import confusion_matrix

In [137]:
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 
           'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome']

df = pd.read_csv('pima-indians-diabetes.csv', names = columns)
df.shape


(768, 9)

In [138]:
#let's check if there are any na values in any column

df.isna().any() 

Pregnancies                 False
Glucose                     False
BloodPressure               False
SkinThickness               False
Insulin                     False
BMI                         False
DiabetesPedigreeFunction    False
Age                         False
Outcome                     False
dtype: bool

In [139]:
df[df == 0].head() # there are zeros in data let's replace them by the column mean

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0
2,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [140]:
for col in df.columns[:-1]:
    df[col].replace(to_replace=0, value=np.mean(df[col]), inplace=True)
    
df.head(10) # replaces zero values by their column means

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6.000000,148.0,72.000000,35.000000,79.799479,33.600000,0.627,50,1
1,1.000000,85.0,66.000000,29.000000,79.799479,26.600000,0.351,31,0
2,8.000000,183.0,64.000000,20.536458,79.799479,23.300000,0.672,32,1
3,1.000000,89.0,66.000000,23.000000,94.000000,28.100000,0.167,21,0
4,3.845052,137.0,40.000000,35.000000,168.000000,43.100000,2.288,33,1
5,5.000000,116.0,74.000000,20.536458,79.799479,25.600000,0.201,30,0
6,3.000000,78.0,50.000000,32.000000,88.000000,31.000000,0.248,26,1
7,10.000000,115.0,69.105469,20.536458,79.799479,35.300000,0.134,29,0
8,2.000000,197.0,70.000000,45.000000,543.000000,30.500000,0.158,53,1
9,8.000000,125.0,96.000000,20.536458,79.799479,31.992578,0.232,54,1


In [141]:
# let's separate the features from labels

X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

X.shape, y.shape

((768, 8), (768,))

In [142]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, random_state = 0)

X_train.shape

(614, 8)

In [143]:
# no need for label encoding for features and labels
# feature scaling

sc_X = StandardScaler()
sc_X.fit_transform(X_train)
sc_X.transform(X_test)

array([[-1.1461866 ,  2.55688505,  0.29184926, ...,  1.47038609,
         2.78594417, -0.96569189],
       [-0.81582693, -0.48558291,  0.12553806, ...,  0.13971749,
        -0.1876381 , -0.88240283],
       [-0.1551076 , -1.51076233, -0.87232915, ...,  0.19695055,
        -0.22668514, -0.71582471],
       ...,
       [-0.1551076 ,  0.67187772,  1.12340527, ...,  1.627777  ,
         0.53623395, -0.96569189],
       [-0.48546727, -0.18795018,  0.12553806, ..., -0.90478581,
        -1.07971278, -0.79911377],
       [-1.1461866 , -0.48558291, -0.04077314, ..., -0.26091391,
         1.06487079, -0.79911377]])

Use 1 output node. Output 0 (<0.5) is considered class A and 1 (>=0.5) is considered class B (in case of sigmoid)

Use 2 output nodes. The input belongs to the class of the node with the highest value/probability (argmax).

In the second case you are probably writing about softmax activation function. If that's true, than the sigmoid is just a special case of softmax function. That's easy to show.

$$y=1/(1+e^−x)=1/(1+1/e^x)=1/((e^x+1)/e^x)=e^x/(1+e^x)=e^x/(e^0+e^x)$$

As you can see sigmoid is the same as softmax. You can think that you have two outputs, but one of them has all weights equal to zero and therefore its output will be always equal to zero.

So the better choice for the binary classification is to use one output unit with sigmoid instead of softmax with two output units, because it will update faster.

In [183]:
EPOCHS = 200
batch_size = 20

def model_baseline():
    model = Sequential()
    model.add(Dense(input_dim = 8, units = 10, activation = 'relu',
                    kernel_initializer = 'uniform'))
    model.add(Dropout(0.1))
    model.add(Dense(units = 10, activation = 'relu',
                    kernel_initializer = 'uniform'))
    model.add(Dropout(0.1))
    model.add(Dense(units = 1, activation = 'sigmoid',
                    kernel_initializer = 'uniform'))
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model    
            
stratified_kfold = StratifiedKFold(n_splits=10)
classifier = KerasClassifier(build_fn = model_baseline, epochs = EPOCHS, batch_size = batch_size)
classifier.fit(X_train, y_train)
              
results = cross_val_score(classifier, X_train, y_train, cv = stratified_kfold, n_jobs = -1, scoring = 'accuracy')
print(results)
print()
print('The mean accuracy is: {}%'.format(np.mean(results) * 100))
print('The variance is: {}'.format(np.std(results)))
              

Epoch 1/200
614/614 [==============================] - 0s 403us/step - loss: 0.6901 - accuracy: 0.5896
Epoch 2/200
614/614 [==============================] - 0s 117us/step - loss: 0.6781 - accuracy: 0.6401
Epoch 3/200
614/614 [==============================] - 0s 120us/step - loss: 0.6760 - accuracy: 0.6401
Epoch 4/200
614/614 [==============================] - 0s 106us/step - loss: 0.6731 - accuracy: 0.6401
Epoch 5/200
614/614 [==============================] - 0s 85us/step - loss: 0.6700 - accuracy: 0.6401
Epoch 6/200
614/614 [==============================] - 0s 78us/step - loss: 0.6685 - accuracy: 0.6401
Epoch 7/200
614/614 [==============================] - 0s 72us/step - loss: 0.6668 - accuracy: 0.6401
Epoch 8/200
614/614 [==============================] - 0s 78us/step - loss: 0.6626 - accuracy: 0.6401
Epoch 9/200
614/614 [==============================] - 0s 85us/step - loss: 0.6614 - accuracy: 0.6466
Epoch 10/200
614/614 [==============================] - 0s 72us/step - loss: 0

614/614 [==============================] - 0s 78us/step - loss: 0.5617 - accuracy: 0.7199
Epoch 81/200
614/614 [==============================] - 0s 85us/step - loss: 0.5601 - accuracy: 0.7199
Epoch 82/200
614/614 [==============================] - 0s 85us/step - loss: 0.5682 - accuracy: 0.7215
Epoch 83/200
614/614 [==============================] - 0s 85us/step - loss: 0.5643 - accuracy: 0.7101
Epoch 84/200
614/614 [==============================] - 0s 78us/step - loss: 0.5533 - accuracy: 0.7199
Epoch 85/200
614/614 [==============================] - 0s 98us/step - loss: 0.5552 - accuracy: 0.7134
Epoch 86/200
614/614 [==============================] - 0s 78us/step - loss: 0.5436 - accuracy: 0.7329
Epoch 87/200
614/614 [==============================] - 0s 85us/step - loss: 0.5609 - accuracy: 0.7036
Epoch 88/200
614/614 [==============================] - 0s 72us/step - loss: 0.5665 - accuracy: 0.7248
Epoch 89/200
614/614 [==============================] - 0s 65us/step - loss: 0.5515 - 

614/614 [==============================] - 0s 72us/step - loss: 0.5422 - accuracy: 0.7378
Epoch 160/200
614/614 [==============================] - 0s 78us/step - loss: 0.5473 - accuracy: 0.7182
Epoch 161/200
614/614 [==============================] - 0s 72us/step - loss: 0.5597 - accuracy: 0.7248
Epoch 162/200
614/614 [==============================] - 0s 85us/step - loss: 0.5422 - accuracy: 0.7215
Epoch 163/200
614/614 [==============================] - 0s 91us/step - loss: 0.5484 - accuracy: 0.7231
Epoch 164/200
614/614 [==============================] - 0s 78us/step - loss: 0.5379 - accuracy: 0.7345
Epoch 165/200
614/614 [==============================] - 0s 72us/step - loss: 0.5374 - accuracy: 0.7394
Epoch 166/200
614/614 [==============================] - 0s 78us/step - loss: 0.5679 - accuracy: 0.7020
Epoch 167/200
614/614 [==============================] - 0s 65us/step - loss: 0.5442 - accuracy: 0.7150
Epoch 168/200
614/614 [==============================] - 0s 91us/step - loss: 

In [202]:
predictions = classifier.predict(X_test)
df_pred = pd.Series(predictions.flatten())
df_actual = pd.Series(y_test)

print(df_pred.value_counts())
df_actual.value_counts()

0    113
1     41
dtype: int64


0    107
1     47
dtype: int64

In [205]:
cm = confusion_matrix(y_test, predictions)

cm_df = pd.DataFrame(data=cm, columns=[['predicted','predicted'], ['Not Diabetic','Diabetic']],
                    index=[['actual','actual'], ['Not Diabetic','Diabetic']])

cm_df

predicted         
                    Not Diabetic Diabetic
actual Not Diabetic           88       19
       Diabetic               25       22